In [0]:
'''!pip uninstall torch torchvision fastai -y
!pip install \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch-1.0.0a0+1d94a2b-cp36-cp36m-linux_x86_64.whl  \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch_xla-0.1+5622d42-cp36-cp36m-linux_x86_64.whl
!pip install torchvision==0.2.0 '''

'!pip uninstall torch torchvision fastai -y\n!pip install   http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch-1.0.0a0+1d94a2b-cp36-cp36m-linux_x86_64.whl    http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch_xla-0.1+5622d42-cp36-cp36m-linux_x86_64.whl\n!pip install torchvision==0.2.0 '

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torchvision import datasets
from skimage import io, transform
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision import transforms
import logging
from datetime import datetime

In [0]:
# Path of Folders
content = '/content/drive/My Drive/new-frames/'
labels_csv = '/content/drive/My Drive/weights.csv'
train_path = content + "train"
test_path = content + "test"
validation_path = content + "validation"


In [0]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

weights=pd.read_csv(labels_csv)
train_IDs, test_IDs = train_test_split(weights, test_size=0.30, random_state=42)

In [0]:
def generateDatasetIDS(list, root_folder = "/content/drive/My Drive/frames/"):
  """
  Return a list with all the image paths relative to those IDs
  """
  test_list = []
  for id in list:
    #print("Id:", id)
    for root, dirs, files in os.walk(root_folder + str(id) ):
        for filename in files:
            #print("\t", str(id) + "/" + filename)
            if('.png' in filename):
              add_file = str(id) + "/" + filename
              test_list.append(add_file)
  return test_list


class CustomListDataset(torch.utils.data.Dataset):

    def __init__(self, images_list, df_weights, root_dir,transform = None):
        """
        Args:
            images_list(list): list with IDs into the dataset
            root_dir(string): directory with all the images
            df_weights(pd_dataframe): dataframe with all the weights of the people
            transform: trasform operation for images
        """
        self.images_list = images_list
        self.root_dir = root_dir
        self.df_weights = df_weights
        self.df_weights = self.df_weights['WEIGHT'].values
        self.transform = transform

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images_list[idx])
        image = Image.open(img_name)
        image = image.convert(mode='RGB')
        
        if(self.transform is not None):
            image = self.transform(image)

        frame_name = self.images_list[idx].split("/")
        id = int(frame_name[0])
        
        labels = np.float(0)
        
        return image, torch.as_tensor(labels)


transform = transforms.Compose([
    transforms.ToTensor()
])



In [0]:
import torch.nn as nn
from torchsummary import summary

class conv2DBatchNormRelu(nn.Module):
    def __init__(
            self,
            in_channels,
            n_filters,
            k_size,
            stride,
            padding,
            bias=True,
            dilation=1,
            with_bn=True,
    ):
        super(conv2DBatchNormRelu, self).__init__()

        conv_mod = nn.Conv2d(int(in_channels),
                             int(n_filters),
                             kernel_size=k_size,
                             padding=padding,
                             stride=stride,
                             bias=bias,
                             dilation=dilation, )

        if with_bn:
            self.cbr_unit = nn.Sequential(conv_mod,
                                          nn.BatchNorm2d(int(n_filters)),
                                          nn.ReLU(inplace=True))
        else:
            self.cbr_unit = nn.Sequential(conv_mod, nn.ReLU(inplace=True))

    def forward(self, inputs):
        outputs = self.cbr_unit(inputs)
        return outputs


class segnetDown2(nn.Module):
    def __init__(self, in_size, out_size):
        super(segnetDown2, self).__init__()
        self.conv1 = conv2DBatchNormRelu(in_size, out_size, 3, 1, 1)
        self.conv2 = conv2DBatchNormRelu(out_size, out_size, 3, 1, 1)
        self.maxpool_with_argmax = nn.MaxPool2d(2, 2, return_indices=True)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.conv2(outputs)
        unpooled_shape = outputs.size()
        outputs, indices = self.maxpool_with_argmax(outputs)
        return outputs, indices, unpooled_shape


class segnetDown3(nn.Module):
    def __init__(self, in_size, out_size):
        super(segnetDown3, self).__init__()
        self.conv1 = conv2DBatchNormRelu(in_size, out_size, 3, 1, 1)
        self.conv2 = conv2DBatchNormRelu(out_size, out_size, 3, 1, 1)
        self.conv3 = conv2DBatchNormRelu(out_size, out_size, 3, 1, 1)
        self.maxpool_with_argmax = nn.MaxPool2d(2, 2, return_indices=True)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.conv2(outputs)
        outputs = self.conv3(outputs)
        unpooled_shape = outputs.size()
        outputs, indices = self.maxpool_with_argmax(outputs)
        return outputs, indices, unpooled_shape


class segnetUp2(nn.Module):
    def __init__(self, in_size, out_size):
        super(segnetUp2, self).__init__()
        self.unpool = nn.MaxUnpool2d(2, 2)
        self.conv1 = conv2DBatchNormRelu(in_size, in_size, 3, 1, 1)
        self.conv2 = conv2DBatchNormRelu(in_size, out_size, 3, 1, 1)

    def forward(self, inputs, indices, output_shape):
        outputs = self.unpool(input=inputs, indices=indices, output_size=output_shape)
        outputs = self.conv1(outputs)
        outputs = self.conv2(outputs)
        return outputs


class segnetUp3(nn.Module):
    def __init__(self, in_size, out_size):
        super(segnetUp3, self).__init__()
        self.unpool = nn.MaxUnpool2d(2, 2)
        self.conv1 = conv2DBatchNormRelu(in_size, in_size, 3, 1, 1)
        self.conv2 = conv2DBatchNormRelu(in_size, in_size, 3, 1, 1)
        self.conv3 = conv2DBatchNormRelu(in_size, out_size, 3, 1, 1)

    def forward(self, inputs, indices, output_shape):
        outputs = self.unpool(input=inputs, indices=indices, output_size=output_shape)
        outputs = self.conv1(outputs)
        outputs = self.conv2(outputs)
        outputs = self.conv3(outputs)
        return outputs


class SegNet(nn.Module):
    def __init__(self, n_classes=3, in_channels=3, is_unpooling=True):
        super(SegNet, self).__init__()

        self.in_channels = in_channels
        self.is_unpooling = is_unpooling

        self.down1 = segnetDown2(self.in_channels, 64)
        self.down2 = segnetDown2(64, 128)
        self.down3 = segnetDown3(128, 256)
        self.down4 = segnetDown3(256, 512)
        self.down5 = segnetDown3(512, 512)

        self.up5 = segnetUp3(512, 512)
        self.up4 = segnetUp3(512, 256)
        self.up3 = segnetUp3(256, 128)
        self.up2 = segnetUp2(128, 64)
        self.up1 = segnetUp2(64, n_classes)

    def forward(self, inputs):

        down1, indices_1, unpool_shape1 = self.down1(inputs)
        down2, indices_2, unpool_shape2 = self.down2(down1)
        down3, indices_3, unpool_shape3 = self.down3(down2)
        down4, indices_4, unpool_shape4 = self.down4(down3)
        down5, indices_5, unpool_shape5 = self.down5(down4)

        up5 = self.up5(down5, indices_5, unpool_shape5)
        up4 = self.up4(up5, indices_4, unpool_shape4)
        up3 = self.up3(up4, indices_3, unpool_shape3)
        up2 = self.up2(up3, indices_2, unpool_shape2)
        up1 = self.up1(up2, indices_1, unpool_shape1)

        return up1


    def encode(self, inputs):
        down1, indices_1, unpool_shape1 = self.down1(inputs)
        down2, indices_2, unpool_shape2 = self.down2(down1)
        down3, indices_3, unpool_shape3 = self.down3(down2)
        down4, indices_4, unpool_shape4 = self.down4(down3)
        down5, indices_5, unpool_shape5 = self.down5(down4)
        
        return down5

    def init_vgg16_params(self, vgg16):
        blocks = [self.down1, self.down2, self.down3, self.down4, self.down5]

        ranges = [[0, 4], [5, 9], [10, 16], [17, 23], [24, 29]]
        features = list(vgg16.features.children())

        vgg_layers = []
        for _layer in features:
            if isinstance(_layer, nn.Conv2d):
                vgg_layers.append(_layer)

        merged_layers = []
        for idx, conv_block in enumerate(blocks):
            if idx < 2:
                units = [conv_block.conv1.cbr_unit, conv_block.conv2.cbr_unit]
            else:
                units = [
                    conv_block.conv1.cbr_unit,
                    conv_block.conv2.cbr_unit,
                    conv_block.conv3.cbr_unit,
                ]
            for _unit in units:
                for _layer in _unit:
                    if isinstance(_layer, nn.Conv2d):
                        merged_layers.append(_layer)

        assert len(vgg_layers) == len(merged_layers)

        for l1, l2 in zip(vgg_layers, merged_layers):
            if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
                assert l1.weight.size() == l2.weight.size()
                assert l1.bias.size() == l2.bias.size()
                l2.weight.data = l1.weight.data
                l2.bias.data = l1.bias.data


In [7]:
import torch.optim as optim

IMAGE_SIZE = 150 * 150
IMAGE_WIDTH = IMAGE_HEIGHT = 150

# Hyperparameters
code_size = 64
num_epochs = 150
batch_size = 128
lr = 0.001

# Load data       train_IDs, test_IDs
train_list = generateDatasetIDS(train_IDs.ID,"/content/drive/My Drive/frames/")
train_custom = CustomListDataset(images_list = train_list, df_weights = weights, root_dir="/content/drive/My Drive/frames/", transform = transform)
train_loader = torch.utils.data.DataLoader(train_custom, batch_size=32, shuffle=True, num_workers=4)

test_list = generateDatasetIDS(test_IDs.ID,"/content/drive/My Drive/frames/")
test_custom = CustomListDataset(images_list = test_list, df_weights = weights, root_dir="/content/drive/My Drive/frames/", transform = transform)
test_loader = torch.utils.data.DataLoader(test_custom, batch_size=32, shuffle=False, num_workers=4)

# Instantiate model
autoencoder = SegNet(n_classes=3).cuda()
optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

autoencoder.train()
# Training loop
for epoch in range(num_epochs):
    print("Epoch %d" % epoch)
    loss = 0.0

    for i, (images, _) in enumerate(train_loader):    # Ignore image labels
        #print(images.shape)
        #img = images)
        img = images.cuda()
        
        optimizer.zero_grad()
        #train_loss = loss_fn(out, img)
        y_hat = autoencoder(img)

        train_loss = torch.sqrt((y_hat - img).pow(2).mean())
        #print(loss)
        train_loss.backward()
        optimizer.step()
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, num_epochs, loss))



Epoch 0
epoch : 1/150, loss = 0.058658
Epoch 1
epoch : 2/150, loss = 0.009946
Epoch 2
epoch : 3/150, loss = 0.006209
Epoch 3
epoch : 4/150, loss = 0.005234
Epoch 4
epoch : 5/150, loss = 0.004768
Epoch 5
epoch : 6/150, loss = 0.004315
Epoch 6
epoch : 7/150, loss = 0.003972
Epoch 7
epoch : 8/150, loss = 0.003742
Epoch 8
epoch : 9/150, loss = 0.003218
Epoch 9
epoch : 10/150, loss = 0.003130
Epoch 10
epoch : 11/150, loss = 0.002940
Epoch 11
epoch : 12/150, loss = 0.002661
Epoch 12
epoch : 13/150, loss = 0.002669
Epoch 13
epoch : 14/150, loss = 0.003017
Epoch 14
epoch : 15/150, loss = 0.002682
Epoch 15
epoch : 16/150, loss = 0.002681
Epoch 16
epoch : 17/150, loss = 0.002962
Epoch 17
epoch : 18/150, loss = 0.002704
Epoch 18
epoch : 19/150, loss = 0.002661
Epoch 19
epoch : 20/150, loss = 0.002662
Epoch 20
epoch : 21/150, loss = 0.002361
Epoch 21
epoch : 22/150, loss = 0.002517
Epoch 22
epoch : 23/150, loss = 0.002407
Epoch 23
epoch : 24/150, loss = 0.002364
Epoch 24
epoch : 25/150, loss = 0.0

In [8]:
import torchvision.models as models
autoencoder = SegNet()
checkpoint = torch.load("/content/drive/My Drive/model/conv_autoencoder_nclasses3-150epoch.pt")
autoencoder.load_state_dict(checkpoint)


FileNotFoundError: ignored

In [15]:
loss = 0
autoencoder.cuda()

for i, (images, _) in enumerate(test_loader):    # Ignore image labels
    #print(images.shape)
    #img = Variable(images)
    img = images.cuda()

    y_hat = autoencoder(img)
    
    test_loss = torch.sqrt((y_hat - img).pow(2).mean())
    # add the mini-batch training loss to epoch loss
    loss += test_loss.item()

# compute the epoch training loss
loss = loss / len(test_loader)

# display the epoch training loss
print("Test Loss = {:.6f}".format(loss))

# Try reconstructing on test data
test_image, _ = test_custom.__getitem__(1)
test_image = test_image.view([1, 3, IMAGE_WIDTH, IMAGE_HEIGHT]).cuda()
test_reconst = autoencoder(test_image)

torchvision.utils.save_image(test_image.data, 'orig1.png')
torchvision.utils.save_image(test_reconst.data, 'reconst1.png')

Test Loss = 0.001571


In [0]:
torch.save(autoencoder.state_dict(), "/content/drive/My Drive/model/conv_autoencoder_test_shuf150it.pt") 
